# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 사용할 변수 선택하기
- 모델링
- 추론


### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [228]:
# basic
import os, random
import pandas as pd
import numpy as np

# model
from catboost import CatBoostRegressor, CatBoostClassifier

# eval metric
from sklearn.metrics import confusion_matrix

# k-fold by timeseries split
from sklearn.model_selection import TimeSeriesSplit

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [229]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [230]:
# load makes data
train = pd.read_csv('../data/train_preprocessed_data.csv')
test = pd.read_csv('../data/test_preprocessed_data.csv')

### 3. 사용할 변수 선택하기

데이터에서 필요한 칼럼을 선택하여 적용하기로 한다.

In [231]:
# onehotencoder in ground
train['A'] = np.where(train['ground'] == 'A', 1, 0)
train['B'] = np.where(train['ground'] == 'B', 1, 0)
train['C'] = np.where(train['ground'] == 'C', 1, 0)
train['D'] = np.where(train['ground'] == 'D', 1, 0)

test['A'] = np.where(test['ground'] == 'A', 1, 0)
test['B'] = np.where(test['ground'] == 'B', 1, 0)
test['C'] = np.where(test['ground'] == 'C', 1, 0)
test['D'] = np.where(test['ground'] == 'D', 1, 0)

In [232]:
# label 선택하기
use_label_x = ['hm', 're', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'A', 'B', 'C', 'D',
       'dew_point', 'sin_time', 'cos_time', 'sin_month', 'cos_month', 'diff_air-dew', 'diff_ts-dew', 'make_copyfog',
       'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'
       ]

use_label_y = ['class']

In [233]:
# train_x, train_y, test_x 만들자
# train: I,J년도, valid: K년도, test: L년도
train_x = train.loc[(train['year'].isin(['I', 'J'])), use_label_x]
train_y = train.loc[(train['year'].isin(['I', 'J'])), use_label_y]

valid_x = train.loc[train['year'].isin(['K']), use_label_x]
valid_y = train.loc[train['year'].isin(['K']), use_label_y]

test_x = test[use_label_x]

### 4. 모델링

모델을 한개만 쓰는것이 더 정확도가 높기 때문에 단순히 Regression, Classification을 비교하였다.  
결과적으로 classification의 결과가 더 좋았다. 따라서 classification 모델을 사용하도록 하자.

Catboost는 시계열 데이터 또는 범주형 데이터에 좋은 성능을 가져오는 모델로 알려져 있기 때문에 적용하도록 한다.

In [234]:
# 선택 모델
cb = CatBoostClassifier(random_state = 42)

In [235]:
# str
train_y = train_y.astype(str)
valid_y = valid_y.astype(str)

In [236]:
# train에 적용
cb.fit(train_x, train_y)

Learning rate set to 0.116899
0:	learn: 1.0671742	total: 1.37s	remaining: 22m 47s
1:	learn: 0.8662088	total: 2.56s	remaining: 21m 20s
2:	learn: 0.7226608	total: 3.62s	remaining: 20m 4s
3:	learn: 0.6133434	total: 4.71s	remaining: 19m 32s
4:	learn: 0.5273469	total: 5.66s	remaining: 18m 47s
5:	learn: 0.4577323	total: 6.71s	remaining: 18m 31s
6:	learn: 0.4004059	total: 7.78s	remaining: 18m 23s
7:	learn: 0.3526418	total: 8.84s	remaining: 18m 16s
8:	learn: 0.3125167	total: 9.85s	remaining: 18m 4s
9:	learn: 0.2783657	total: 10.7s	remaining: 17m 44s
10:	learn: 0.2492771	total: 11.6s	remaining: 17m 25s
11:	learn: 0.2244282	total: 12.5s	remaining: 17m 7s
12:	learn: 0.2027980	total: 13.6s	remaining: 17m 15s
13:	learn: 0.1840815	total: 14.9s	remaining: 17m 29s
14:	learn: 0.1678159	total: 16.1s	remaining: 17m 37s
15:	learn: 0.1536417	total: 17.3s	remaining: 17m 45s
16:	learn: 0.1412601	total: 18.5s	remaining: 17m 51s
17:	learn: 0.1305590	total: 19.6s	remaining: 17m 51s
18:	learn: 0.1211718	total: 2

In [237]:
# predict value
cb_pred = cb.predict(valid_x)

In [238]:
# evaluation
def csi_score(CONFUSION_MATRIX, exceptcol = 0):

    # 차원의 수
    n, _ = CONFUSION_MATRIX.shape

    # 계산하여 받을 값
    H = 0
    F = 0
    M = 0
    for i in range(n):
        for j in range(n):
            if i == j == exceptcol:
                continue
            elif i == j:
                H += CONFUSION_MATRIX[i][j]
            elif j != exceptcol:
                F += CONFUSION_MATRIX[i][j]
            elif i != exceptcol:
                M += CONFUSION_MATRIX[i][j]

    return H / (H + F + M)

In [239]:
csi_score(confusion_matrix(valid_y, cb_pred), 3)

0.05019526763151849

### 5. 추론

- train, valid를 모두 포함한 데이터를 통해 전체적으로 학습을 다시 진행한 다음 test를 적합하도록 한다.  
- 이름을 꼭 ID에 맞도록 설정해 주어야 한다.